In [1]:
import math
import random

In [2]:
class Value:
  def __init__(self, inData, inOperation = "", inChildren = None, inLabel = ""):
    self.data = float(inData)
    self.grad = 0.0
    self.operation = inOperation
    self.nBackward = lambda: None
    self.label = inLabel

    
    self.children = []
    if (inChildren is not None):
      self.children = [children for children in inChildren]
    
    
  def __repr__(self):
    return f"Value[{self.label}]({self.data}), {self.operation if self.operation != '' else 'none'}, ( {len(self.children)} ), grad = {self.grad}"

  # Operators

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, "add", [self, other])

    def nBackward():
      self.grad += 1 * out.grad
      other.grad += 1 * out.grad

    out.nBackward = nBackward  
    return out

  def __radd__(self, other):
    return self + other
  
  def __neg__(self):
    out = Value(- self.data, "neg", [self])

    def nBackward():
      self.grad += -1 * out.grad

    out.nBackward = nBackward
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, "mul", [self, other])
    
    def nBackward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out.nBackward = nBackward
    return out

  def __rmul__(self, other):
    return self * other

  def __pow__(self, inPow):
    out = Value(self.data ** inPow, "pow", [self])

    def nBackward():
      self.grad += inPow * (self.data ** (inPow - 1)) * out.grad

    out.nBackward = nBackward
    
    return out

  def __truediv__ (self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data / other.data, "div", [self, other])

    def nBackward():
      self.grad += 1/other.data * out.grad
      other.grad += self.data * -1 * (other.data**-2) * out.grad

    out.nBackward = nBackward
    return out

  def __rtruediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    return other / self

  def tanh(self):
    out = Value(math.tanh(self.data), "tanh", [self])

    def nBackward():
      self.grad += (1 - math.tanh(self.data)**2) * out.grad
    
    out.nBackward = nBackward
    return out
    
  def backward(self):
    graph = []
    visited = set()
  
    def buildPath(inNode):

      if (inNode not in visited):
        visited.add(inNode)
        for child in inNode.children:
          buildPath(child)
          
        graph.append(inNode)

    buildPath(self)
    self.grad = 1.0

    for it in reversed(graph):
      it.nBackward()

  def zeroGrad(self):
    visited = set()
    
    def clear(inNode):
      if inNode not in visited:
        visited.add(inNode)
        inNode.grad = 0.0
        
        for child in inNode.children:
          zeroGrad(child)

    clear(self)

In [20]:
class Neuron:
  def __init__(self, inNNumber):
    self.bias = Value(random.uniform(-1.0, 1.0))
    self.weights = [Value(random.uniform(-1.0, 1.0)) for inN in range(inNNumber)]

  def __call__(self, inX):
    newSum = sum(wi * xi for wi, xi in zip(self.weights, inX)) + self.bias
    
    out = newSum.tanh()
    return out
    
class Layer:
  def __init__(self, inNeurons, outNeurons):
    self.neurons = [Neuron(inNeurons) for neuron in range (outNeurons)]
    
  def __call__(self, inX):
    out = [neuron(inX) for neuron in self.neurons]
    return out

class MLP:
  def __init__(self, layerInfo):
    self.network = []

    for layer in layerInfo:
      self.network.append(Layer(layer["in"], layer["out"]))

  def __call__(self, inX):
    x = inX
    for layer in self.network:
      x = layer(x)

    return x

In [21]:
def printValues(inValue):
  
  valuesSet = set()

  def getValues(inValue):
    valuesSet.add(inValue)
    for value in inValue.children:
      getValues(value)

  getValues(inValue)
  for value in valuesSet:
    print(f"{value.data} - [{value.label}] - GRAD: {value.grad}")

In [22]:
# Grad test

a = Value(3.0, inLabel = "a")
b = Value(2.0, inLabel = "b")

c = a + b # 5
c.label = "c"

#d = 
c.backward()

print("#############")
print(c)
printValues(c)

#############
Value[c](5.0), add, ( 2 ), grad = 1.0
2.0 - [b] - GRAD: 1.0
5.0 - [c] - GRAD: 1.0
3.0 - [a] - GRAD: 1.0


In [25]:
# Neuron test
neuronTest = Neuron(1)
print("Neuron\n", neuronTest([3]))

# Layer test
layerTest = Layer(3, 2)
print("Layer:\n", layerTest([3, 1, 2]))

# Network test
networkTest = MLP([
                    {"in": 3, "out": 5},
                    {"in": 5, "out": 5},
                    {"in": 5, "out": 5},
                    {"in": 5, "out": 5},
                    {"in": 5, "out": 3}
                  ])
print("Network:\n",networkTest([3.0, 2.0, 1.0]))

Neuron
 Value[](-0.872125782709308), tanh, ( 1 ), grad = 0.0
Layer:
 [Value[](-0.9901167614120889), tanh, ( 1 ), grad = 0.0, Value[](0.4279582705688546), tanh, ( 1 ), grad = 0.0]
Network:
 [Value[](0.8628866618193094), tanh, ( 1 ), grad = 0.0, Value[](-0.9196454075696832), tanh, ( 1 ), grad = 0.0, Value[](0.2316756931923137), tanh, ( 1 ), grad = 0.0]
